# Mummies

In [37]:
import numpy as np
import pandas as pd

In [8]:
pd.set_option('display.max_columns', 30)

## Loading data

In [61]:
mummies_df = pd.read_excel('data/Combined%20Mummy%20Database.xlsx', header=1)
mummies_df.drop('Unnamed: 4', inplace=True, axis=1)
mummies_df.drop('Unnamed: 25', inplace=True, axis=1)
mummies_df.fillna('', inplace=True)

In [62]:
mummies_df.shape

(960, 24)

In [63]:
mummies_df.head()

,Name,Impact Number,Institution,Contact person?,Designation,Animal,Human,Male,Female,Subadult,Adult,Lower Age,Upper Age,Lower Date,Upper Date,Dynasty,Period,Status,site,reference,xray?,CT?,comments,pathology
0,Male Mummy Kiev,NON,Pechersk Lavra,Dr. Mykola Teresenko,,,y,y,,,y,,,11th C BCE,10th C BCE,,,,,http://www.express.co.uk/news/world/806511/egy...,,,elite burial style,
1,Crocodile,NON,Pechersk Lavra,Dr. Mykola Teresenko,,y,,,,,,,,,,,,,,http://www.express.co.uk/news/world/806511/egy...,,,161cm,
2,Menat,NON,Pechersk Lavra,Dr. Mykola Teresenko,,,y,,y,,y,,,,,,,,,http://www.express.co.uk/news/world/806511/egy...,,,post-mortem decapitation,
3,Neswaiu,NON,Museum of Meditarranean and Near Eastern Antiq...,,,,y,,,,,50,60,3rd c BC,,,,,,http://3dprint.com/60734/3d-scanned-printed-mu...,,yes,also 3d printed,
4,Nesmen Khetamun,NON,Shrewsbury Museum & Art Gallery,,,,y,,,,,40,,600bc,,,,,,http://www.shropshirestar.com/news/2015/01/24/...,,,,


## Transforming data

### Unique name

In [64]:
counts = mummies_df.Name.value_counts()

In [65]:
for i, row in mummies_df.iterrows():
    if not row.Name:
        name = 'Unknown ---' + str(i)
    elif counts[row.Name] == 1:
        name = row.Name.strip()
    else:
        name = row.Name.strip() + ' ---' + str(i)
    mummies_df.loc[i, 'UNIQUE_NAME'] = name

### Institution

In [66]:
mummies_df.Institution.nunique()

260

In [83]:
institutions_dict = {
    '': '',
    ' Fondazione Museo delle Antichità Egizie': 'Fondazione Museo delle Antichità Egizie',
    '(?) Museum of Fine Arts Houston': 'Museum of Fine Arts Houston',
    '(Tomb 1 & 2)': 'UNKNOWN',
    '08-480.2a-c': 'Brooklyn Museum',
    '11.6': 'Brooklyn Museum',
    '52.128a-e': 'Brooklyn Museum',
    '8898*88**': 'Garrett Evangelical Theological Seminary (Seabury-Western), Illinois',
    'Academy of Natural Sciences of Drexel University': 'Academy of Natural Sciences of Drexel University',
    'Aksaray Müzesi': 'Aksaray Museum',
    'Albany Institute of History & Art': 'Albany Institute of History and Art',
    'Albert Hall Museum': 'Albert Hall Museum',
    'Amasya Museum Hall of Mummies': 'Amasya Museum',
    'Archaeological Museum in Zagreb': 'Archaeological Museum in Zagreb',
    'Archaeological Museum of Zanjan': 'Archaeological Museum of Zanjan',
    'Assaraya Alhamra Museum': 'Assaraya Alhamra Museum',
    'Australia Museum': 'Australian Museum',
    'Australian Museum': 'Australian Museum',
    'Barnum Museum': 'Barnum Museum',
    'Basel University': 'Basel University',
    'Bavarian State Archaeological Collection': 'Bavarian State Archaeological Collection',
    'Berkshire Museum': 'Berkshire Museum',
    'Bolton Museum': 'Bolton Museum',
    'Boston Museum of Fine Arts': 'Museum of Fine Arts Boston',
    'British Museum': 'British Museum',
    'Brooklyn Museum': 'Brooklyn Museum',
    'Buffalo Museum of Science': 'Buffalo Museum of Science',
    'Burke Museum': 'Burke Museum',
    'CG24100': 'Egyptian Museum',
    'CG33281': 'Egyptian Museum',
    'Cairo Museum The Virtual Eqyptian Museum': 'Eqyptian Museum',
    'Caldwell Heritage Museum': 'Caldwell Heritage Museum',
    'Canadian Museum of Civilization': 'Canadian Museum of Civilization',
    'Canterbury Museum, NZ': 'Canterbury Museum, NZ',
    'Carnegie Museum': 'Carnegie Museum',
    'Centre for the Study of the Arctic;Shemanovsky Science Museum and Exhibition Centre': 'Shemanovsky Science Museum and Exhibition Centre',
    'Chatham-Kent Museum': 'Chatham-Kent Museum',
    'Church of Kampehl': 'Church of Kampehl',
    'Church of San Frediano': 'Church of San Frediano',
    'Cincinnati Museum Center': 'Cincinnati Museum Center',
    'Civic Museum of Ligurian Archaeology, Genoa': 'Civic Museum of Ligurian Archaeology, Genoa',
    'Complexual National Muzeal ASTRA': 'Complexual National Muzeal ASTRA',
    'Corpus Domini Bologna': 'Corpus Domini Bologna',
    'Denvery Museum of Nature and Science': 'Denver Museum of Nature and Science',
    'Department of Egyptian Antiquities of the British Museum': 'British Museum',
    'Derby Museum': 'Derby Museum',
    'Detroit Institute of Art': 'Detroit Institute of Art',
    'Drents Museum': 'Drents Museum',
    'Druban Museum': 'Durban Museum',
    'EA21809': 'British Museum',
    'EA21811': 'British Museum',
    'EA6713': 'British Museum',
    'Ecuadorian Museum': 'Ecuadorian Museum',
    'Egypt Centre - Canolfan Eiffraidd, Swansea': 'Egypt Centre - Canolfan Eiffraidd, Swansea',
    'Egyptian Museum': 'Egyptian Museum',
    'Egyptian Museum ': 'Egyptian Museum',
    'Egyptian Museum of Antiquities': 'Egyptian Museum',
    'Egyptian Museum, Torino Italy': 'Egyptian Museum, Turin Italy',
    'Egyptian Museum, Turin Italy': 'Egyptian Museum, Turin Italy',
    'Egyptian National Museum of Antiquities, Cairo': 'Egyptian Museum',
    'Eternal Egypt': 'British Museum',
    'Eve-Maria Zimmermann Collection, Tenerife REM, Mannheim Germany': 'Eve-Maria Zimmermann Collection',
    'Field Museum of National History, Chicago': 'Field Museum (Chicago)',
    'Fleming Museum': 'Fleming Museum',
    'Fondazione Museo delle Antichità Egizie': 'Fondazione Museo delle Antichità Egizie',
    'Garstang Museum of Archaeology Liverpool': 'Garstang Museum of Archaeology Liverpool',
    'Geophysical Institute University of Alaska Fairbanks': 'Geophysical Institute University of Alaska Fairbanks',
    'Grand Rapids Public Museum': 'Grand Rapids Public Museum',
    'Hancock Museum': 'Grand Rapids Public Museum',
    'Haslemere Museum': 'Grand Rapids Public Museum',
    'Historisches Museum Bern CH': 'Museum of History and Ethnography, St. Gallen, Switzerland',
    'Historisches Museum urg, CHAargau, Schloss Lenz': 'Museum of History and Ethnography, St. Gallen, Switzerland',
    'Historisches und Volkerkundemuseum': 'Museum of History and Ethnography, St. Gallen, Switzerland',
    'Historisches und Volkerkundemuseum, St. Gallen CH': 'Museum of History and Ethnography, St. Gallen, Switzerland',
    'Huando Museum': 'Huando Museum',
    'Hunan Provincial Museum': 'Hunan Provincial Museum',
    'Hungarian National History Museum': 'Hungarian National History Museum',
    'Israel Museum': 'Israel Museum',
    'Iziko Museum': 'Iziko Museum',
    'J. Paul Getty Museum': 'J. Paul Getty Museum',
    'JB Getty Museum': 'J. Paul Getty Museum',
    'JE26206(b)': 'Egyptian Museum',
    'JE26216(b)': 'Egyptian Museum',
    'JE26218(b)+B47': 'Egyptian Museum',
    'JE26225(b)': 'Egyptian Museum',
    'JE56265(b)': 'Egyptian Museum',
    'JE59130(b)': 'Egyptian Museum',
    'Joseon Dynasty Human Sample Collection of Seol National University College of Medicine': 'Seoul National University',
    'KNH Centre for Biomedical Egyptology': 'KNH Centre for Biomedical Egyptology',
    'Kalamazoo Valley Museum': 'Kalamazoo Valley Museum',
    'Karaman Museum, Turkey': 'Karaman Museum, Turkey',
    'Kemper Art Museum': 'Kemper Art Museum',
    'Kingdom of Permafrost on Nov 17/15': 'Kingdom of Permafrost on Nov 17/15',
    'Kulturhistorisk Museum Oslo': 'Kulturhistorisk Museum Oslo',
    'Kunsthistorishces Museum': 'Kunsthistorisches Museum',
    'L33-002-006': 'Museum of World Treasures',
    'L53-001-006': 'Museum of World Treasures',
    'Leeds Museum': 'Leeds Museum',
    'Leicester Museum New Walk Museum & Art Gallery': 'Leicester Museum New Walk Museum & Art Gallery',
    'Leiden Museum': 'Leiden Museum',
    'Lippisches Landes Museum': 'Lippisches Landes Museum',
    'Liverpool Museums': 'Liverpool Museums',
    'Louvre, Paris': 'Louvre, Paris',
    'MMA 11.139': 'Metropolitan Museum of Art',
    'MMA 11.150.15c': 'Metropolitan Museum of Art',
    'MMA 12.182.131c': 'Metropolitan Museum of Art',
    'MMA 12.182.48c': 'Metropolitan Museum of Art',
    'MMA 25.3.219': 'Metropolitan Museum of Art',
    'MMA 26.3.12': 'Metropolitan Museum of Art',
    'MMA 86.1.35': 'Metropolitan Museum of Art',
    'MMA 86.1.51': 'Metropolitan Museum of Art',
    'MMA 99.3.5': 'Metropolitan Museum of Art',
    'Mabee-Gerrer Museum of Art': 'Mabee-Gerrer Museum of Art',
    'Maidstone Museum': 'Maidstone Museum, England',
    'Maidstone Museum, England': 'Maidstone Museum, England',
    'Manchester Museum': 'Manchester Museum',
    'Marischal Museum': 'Marischal Museum',
    'McClung Museum courtesy of Peabody Museum of Archaeology and Ethnology': 'McClung Museum',
    'McClung Museum purchased from Toledo Museum of Art 1991.7.144': 'McClung Museum',
    'McGill University': 'McGill University',
    'Medelhavsmuseet': 'Medelhavsmuseet',
    'Michael C. Carlos Museum': 'Michael C. Carlos Museum',
    'Milwaukee Public Museum': 'Milwaukee Public Museum',
    'Minneapolis Institute of Art': 'Minneapolis Institute of Art',
    'Monastery of Attel, Upper Bavaria': 'Monastery of Attel, Upper Bavaria',
    'Musee Royaux, Brussels': 'Musee Royaux, Brussels',
    "Musee d'Ethnographie": "Musee d'Ethnographie",
    "Musee d'Orbe, Switzerland": "Musee d'Orbe, Switzerland",
    "Musee d'Yverdon et region, Yverdon CH": "Musee d'Yverdon et region, Yverdon CH",
    'Musee de la Civilisation': 'Musee de la Civilisation',
    'Musei Vaticani': 'Musei Vaticani',
    'Musem of Fine Arts Boston': 'Museum of Fine Arts Boston',
    'Museo Nacional de Antropologia, Biodiversidad Agricultura y Alimentacion (MUNABA)': 'Museo Nacional de Antropologia, Biodiversidad Agricultura y Alimentacion (MUNABA)',
    'Museo Nacional, Lima': 'Museo Nacional, Lima',
    'Museo Santuarios de Altura': 'Museo Santuarios de Altura',
    'Museo de El Carmen': 'Museo de El Carmen',
    'Museo de la Memoria los Derechos Humanos': 'Museo de la Memoria los Derechos Humanos',
    'Museo di Anatomia Umana, Universita di Pisa': 'Museo di Anatomia Umana, Universita di Pisa',
    'Museum Victoria': 'Museum Victoria',
    'Museum Yverdon und Region, Yverdon-les-bains': 'Museum Yverdon und Region, Yverdon-les-bains',
    'Museum of Egyptian Antiquities, Swansea University': 'Museum of Egyptian Antiquities, Swansea University',
    'Museum of Fine Arts Boston': 'Museum of Fine Arts Boston',
    'Museum of History and Ethnography, St. Gallen, Switzerland': 'Museum of History and Ethnography, St. Gallen, Switzerland',
    'Museum of Man San Diego': 'Museum of Man San Diego',
    'Museum of Meditarranean and Near Eastern Antiquities (Stockholm)': 'Museum of Meditarranean and Near Eastern Antiquities (Stockholm)',
    'Museum of Science': 'Museum of Science',
    'Museum of Taizhou': 'Museum of Taizhou',
    'Museum of Witchcraft': 'Museum of Witchcraft',
    'Museum of World Treasures': 'Museum of World Treasures',
    'Mutter Museum': 'Mutter Museum',
    'Myers Museum, Eton College': 'Myers Museum, Eton College',
    'National Cultural History Museum': 'National Cultural History Museum',
    'National Museum of Antiquities': 'National Museum of Antiquities',
    'National Museum of Archaeology, Lisbon Portugal': 'National Museum of Archaeology, Lisbon Portugal',
    'National Museum of Ireland': 'National Museum of Ireland',
    'National Museum of Natural History (Smithsonian Institution; USNM),': 'National Museum of Natural History',
    'National Museum of Natural History Department of Anthropology P381235': 'National Museum of Natural History',
    'National Museum of Scotland': 'National Museum of Scotland',
    'Nationalmuseet, Copenhagen DK': 'Nationalmuseet, Copenhagen DK',
    'Natural History Museum': 'Natural History Museum',
    'Natural History Museum of Los Angeles': 'Natural History Museum of Los Angeles',
    'Natural History Museum, Basel': 'Natural History Museum, Basel',
    'Natural History Museum, Vienna': 'Natural History Museum, Vienna',
    'Naturwissenschaftliche Sammlungen Winterthur': 'Naturwissenschaftliche Sammlungen Winterthur',
    'Nicholson Museum': 'Nicholson Museum University of Sydney',
    'Nicholson Museum University of Sydney': 'Nicholson Museum University of Sydney',
    'Norfolk Museums': 'Norfolk Museums',
    'Ohio History Connection': 'Ohio History Connection',
    'Old Capital Museum': 'Old Capital Museum',
    'Otago Museum': 'Otago Museum',
    'PV.2010.44': 'UNKNOWN',
    'PV.2010.45': 'UNKNOWN',
    'Peabody Museum': 'Peabody Museum',
    'Peabody?': 'Peabody Museum',
    'Pechersk Lavra': 'Pechersk Lavra',
    'Penn Museum': 'Penn Museum',
    'Perth Museum and Art Gallery': 'Perth Museum and Art Gallery',
    'Queensland Museum': 'Queensland Museum',
    'REM, Mannheim, Germany': 'REM, Mannheim, Germany',
    'RISD Museum': 'RISD Museum',
    'Ratisches Museum Chur CH': 'Ratisches Museum Chur CH',
    'Raymond Dart Collection': 'Raymond Dart Collection',
    'Redpath Museum': 'Redpath Museum',
    'Reiss-Engelhorn-Museen, Mannheim DE': 'Reiss-Engelhorn-Museen, Mannheim DE',
    'Rijksmuseum van Oudheden': 'Rijksmuseum van Oudheden',
    'Robert Hull Fleming Museum': 'Robert Hull Fleming Museum',
    'Rosicrucian Egyptian Museum': 'Rosicrucian Egyptian Museum',
    'Royal Ontario Museum': 'Royal Ontario Museum',
    'Rueil-Malmaison museum': 'Rueil-Malmaison museum',
    'Rutgers Geology Museum on loan from New Brunswick Theological Seminary': 'Rutgers Geology Museum on loan from New Brunswick Theological Seminary',
    'Ryusui-ji Dainchibou Temple': 'Ryusui-ji Dainchibou Temple',
    'SR4.11326(b)': 'UNKNOWN',
    'Saint Louise Art Museum': 'Saint Louise Art Museum',
    'San Francisco Fine Arts Museum': 'San Francisco Fine Arts Museum',
    'San Fransico State University Museum': 'San Fransico State University Museum',
    'Schloss Gottorf, Schleswig DE': 'Schloss Gottorf, Schleswig DE',
    'Shemanovsky Museum': 'Shemanovsky Museum',
    'Shrewsbury Museum & Art Gallery': 'Shrewsbury Museum & Art Gallery',
    'Society of Vertebrate Paleontology': 'Society of Vertebrate Paleontology',
    'South Australia Museum': 'South Australia Museum',
    'South Australian Museum': 'South Australian Museum',
    'South Tyrol Museum of Archaeology': 'South Tyrol Museum of Archaeology',
    'Southern Baptist Theological Seminary': 'Southern Baptist Theological Seminary',
    'Speed Art Museum': 'Speed Art Museum',
    'Spurlock Museum University of Illinois': 'Spurlock Museum University of Illinois',
    "St. Michan's Church": "St. Michan's Church",
    'St. Petersburg Museum of History': 'St. Petersburg Museum of History',
    'Staatliche Kunstsammlungen Dresed': 'Staatliche Kunstsammlungen Dresed',
    'State Archaeology Museum (India)': 'State Archaeology Museum (India)',
    'State Museum, India': 'State Museum, India',
    'Stiftsbibliothe, St. Gallen': 'Stiftsbibliothe, St. Gallen',
    'Swansea Museum U.K.': 'Swansea Museum U.K.',
    'TR13.3.15.1(d)': 'UNKNOWN',
    'TR13.3.15.1(f)': 'UNKNOWN',
    'TR13.3.15.1€': 'UNKNOWN',
    'Tasmanian Museum & Art Gallery': 'Tasmanian Museum & Art Gallery',
    'Te Papa (prev. Colonial Museum)': 'Te Papa (prev. Colonial Museum)',
    'Te Papa Museum': 'Te Papa (prev. Colonial Museum)',
    'The Art Institute of Chicago': 'The Art Institute of Chicago',
    'The British Museum': 'British Museum',
    'The Field Museum': 'Field Museum (Chicago)',
    'The Field Museum (Chicago)': 'Field Museum (Chicago)',
    'The Fitzwilliam Museum': 'Fitzwilliam Museum',
    'The Global Egyptian Museum/Kunsthistorisches Museum': 'Global Egyptian Museum',
    'The Kelsey Museum of Archaeology': 'Kelsey Museum of Archaeology',
    'The Metropolitan Museum of Art': 'Metropolitan Museum of Art',
    'The Oriental Insitute University of Chicago': 'Oriental Insitute University of Chicago',
    'The Walters Art Museum': 'Walters Art Museum',
    'The Walters Art Museum on loan from Goucher College, Baltimore': 'Walters Art Museum',
    'Tokyo National Museum': 'Tokyo National Museum',
    'Torquay Museum': 'Torquay Museum',
    'Tulane': 'Tulane',
    'UPM L-55-15A': 'University of Pennsylvania Museum',
    'UiO Museum of Cultural Histor': 'Museum of Cultural History, Oslo',
    'UiO Museum of Cultural History': 'Museum of Cultural History, Oslo',
    'Ulster Museum': 'Ulster Museum',
    'Universalmuseeum Joanneum Graz AU': 'Universalmuseeum Joanneum Graz AU',
    'Universite de Quebec a Montreal': 'Universite de Quebec a Montreal',
    'University Museums of Padua': 'University Museums of Padua',
    'University of Alaska Musem of the North': 'University of Alaska Musem of the ',
    'University of Pennsylvania Museum of Archaeology and Anthropology (UPM),': 'University of Pennsylvania Museum',
    'University of Pisa': 'University of Pisa',
    'University of Richmond': 'University of Richmond',
    'University of Sydney': 'University of Sydney',
    'University of Western Ontario': 'University of Western Ontario',
    'University of Wyoming': 'University of Wyoming',
    'Vatican Museum': 'Vatican Museum',
    'Victor Balaguer Library Museum': 'Victor Balaguer Library Museum',
    'Virginia Museum of Fine Arts': 'Virginia Museum of Fine Arts',
    'Volkerkundemuseum Basel CH': 'Volkerkundemuseum Basel CH',
    'Von Luschan & American Museum of Natural History (AMNH)': 'American Museum of Natural History',
    'Wayne County Historical Museum': 'Wayne County Historical Museum',
    'Westminster College': 'Westminster College',
    'Williams College Musem of Art': 'Williams College Musem of Art',
    'World Museum Liverpool': 'World Museum Liverpool',
    'Xingjiang Museum': 'Xingjiang Museum',
    'Yale Peabody': 'Yale Peabody',
    'de Young Museum': 'de Young Museum',
    'skiff': 'UNKNOWN',
    'wnichols@museumofvancouver.ca': 'UNKNOWN',
}

In [ ]:
institutions_locs = { # lat, lng
    '': 'UNKNOWN',
    'Academy of Natural Sciences of Drexel University': (39.957, -75.1714),
    'Aksaray Museum': (38.361111, 33.993611),
    'Albany Institute of History and Art': (42.655774, -73.760372),
    'Albert Hall Museum': (26.91179, 75.81953),
    'Amasya Museum': (40.6483985,35.8227312),
    'American Museum of Natural History': (40.780556, -73.974722),
    'Archaeological Museum in Zagreb': (45.811111, 15.977222),
    'Archaeological Museum of Zanjan': (36.6709199,48.4784561),
    'Assaraya Alhamra Museum': (32.895833, 13.180278),
    'Australian Museum': (-33.877432, 151.212732),
    'Barnum Museum': (41.175278, -73.188611),
    'Basel University': (47.55858, 7.5836),
    'Bavarian State Archaeological Collection': (48.143889, 11.591111),
    'Berkshire Museum': (42.4473, -73.2536),
    'Bolton Museum': (53.5774, -2.431557),
    'British Museum': (51.519459, -0.126931),
    'Brooklyn Museum': (40.671306, -73.96375),
    'Buffalo Museum of Science': (42.906132, -78.843314),
    'Burke Museum': (47.660972, -122.309778),
    'Caldwell Heritage Museum': (35.9116091,-81.5385798),
    'Canadian Museum of Civilization': (45.429722, -75.708889),
    'Canterbury Museum, NZ': (-43.5309, 172.6271),
    'Carnegie Museum': (40.44369, -79.948976),
    'Shemanovsky Science Museum and Exhibition Centre': (66.5359062,66.607585),
    'Chatham-Kent Museum': (42.3891543,-82.151762),
    'Church of Kampehl': (52.866667, 12.433056),
    'Church of San Frediano': (43.846111, 10.504722),
    'Cincinnati Museum Center': (39.11, -84.537778),
    'Civic Museum of Ligurian Archaeology, Genoa': (44.4291656,8.8126433),
    'Complexual National Muzeal ASTRA': (45.7975571,24.1510981),
    'Corpus Domini Bologna': (44.489167, 11.339722),
    'Denver Museum of Nature and Science': (39.7475, -104.9425),
    'Derby Museum': (38.2053, -85.7712),
    'Detroit Institute of Art': (42.359292, -83.064797),
    'Drents Museum': (52.9933, 6.5644),
    'Druban Museum': (-29.883333, 31.05),
    'Ecuadorian Museum': (-0.233333, -78.516667),
    'Egypt Centre - Canolfan Eiffraidd, Swansea': (51.6101373,-3.9788101),
    'Egyptian Museum': (30.047778, 31.233333),
    'Egyptian Museum, Turin Italy': (45.068333, 7.684444),
    'Eve-Maria Zimmermann Collection': 'UNKNOWN',
    'Field Museum (Chicago)': (41.86611, -87.61694),
    'Fitzwilliam Museum': (52.200278, 0.119444),
    'Fleming Museum': (44.48059, -73.199),
    'Fondazione Museo delle Antichità Egizie': (45.068333, 7.684444),
    'Garrett Evangelical Theological Seminary (Seabury-Western), Illinois': (42.0562, -87.6755),
    'Garstang Museum of Archaeology Liverpool': (53.4022133,-2.9644639),
    'Geophysical Institute University of Alaska Fairbanks': (64.859444, -147.849722),
    'Global Egyptian Museum': 'UNKNOWN',
    'Grand Rapids Public Museum': (42.965528,-85.676715),
    'Huando Museum': (-11.486175,-77.1836962),
    'Hunan Provincial Museum': (28.2153, 112.9878),
    'Hungarian National History Museum': (47.482222, 19.085556),
    'Israel Museum': (31.772379, 35.204524),
    'Iziko Museum': (-33.928889, 18.415),
    'J. Paul Getty Museum': (34.0775, -118.475),
    'Seoul National University': (37.46, 126.95),
    'KNH Centre for Biomedical Egyptology': (53.4668498,-2.2360777),
    'Kalamazoo Valley Museum': (42.29329, -85.58392),
    'Karaman Museum, Turkey': (37.182778, 32.211389),
    'Kelsey Museum of Archaeology': (42.276861, -83.740556),
    'Kemper Art Museum': (38.6471177,-90.3027034),
    'Kingdom of Permafrost on Nov 17/15': 'UNKNOWN',
    'Kulturhistorisk Museum Oslo': (59.9167934,10.735523),
    'Kunsthistorishces Museum': (48.2037, 16.3614),
    'Leeds Museum': (53.801617, -1.547323),
    'Leicester Museum New Walk Museum & Art Gallery': (52.628954, -1.127765),
    'Leiden Museum': (52.1270298,3.0515736),
    'Lippisches Landes Museum': (51.93715,8.874576),
    'Liverpool Museums': (53.4185418,-3.0250408),
    'Louvre, Paris': (48.8606111,2.337644),
    'Mabee-Gerrer Museum of Art': (35.366848, -96.953704),
    'Maidstone Museum, England': (51.27602,0.520466),
    'Manchester Museum': (53.4766765,-2.2954527),
    'Marischal Museum': (57.1489883,-2.0993574),
    'McClung Museum': (35.951999, -83.927178),
    'McGill University': (45.504167, -73.574722),
    'Medelhavsmuseet': (59.279511, 18.067467),
    'Metropolitan Museum of Art': (40.77891, -73.96367),
    'Michael C. Carlos Museum': (33.790278, -84.324167),
    'Milwaukee Public Museum': (43.040744, -87.921095),
    'Minneapolis Institute of Art': (44.958611, -93.274167),
    'Monastery of Attel, Upper Bavaria': (47.5698601,11.0951279),
    'Musee Royaux, Brussels': (50.8419778,4.3579037),
    "Musee d'Ethnographie": (46.197946, 6.136959),
    "Musee d'Orbe, Switzerland": (46.7233168,6.5325746),
    "Musee d'Yverdon et region, Yverdon CH": (46.7783713,6.6416352),
    'Musee de la Civilisation': (46.815108, -71.202383),
    'Musei Vaticani': (41.906389, 12.454444),
    'Museo Nacional de Antropologia, Biodiversidad Agricultura y Alimentacion (MUNABA)': (-12.0737506,-77.0392978),
    'Museo Nacional, Lima': (-12.077222, -77.061944),
    'Museo Santuarios de Altura': (-16.3995748,-71.5376988),
    'Museo de El Carmen': (19.345, -99.1894),
    'Museo de la Memoria los Derechos Humanos': (33.439839, -70.679375),
    'Museo di Anatomia Umana, Universita di Pisa': (43.714596,10.3947562),
    'Museum Victoria': (),
    'Museum Yverdon und Region, Yverdon-les-bains': (),
    'Museum of Cultural History, Oslo': (),
    'Museum of Egyptian Antiquities, Swansea University': (),
    'Museum of Fine Arts Boston': (),
    'Museum of Fine Arts Houston': (),
    'Museum of History and Ethnography, St. Gallen, Switzerland': (),
    'Museum of Man San Diego': (),
    'Museum of Meditarranean and Near Eastern Antiquities (Stockholm)': (),
    'Museum of Science': (),
    'Museum of Taizhou': (),
    'Museum of Witchcraft': (),
    'Museum of World Treasures': (),
    'Mutter Museum': (),
    'Myers Museum, Eton College': (),
    'National Cultural History Museum': (),
    'National Museum of Antiquities': (),
    'National Museum of Archaeology, Lisbon Portugal': (),
    'National Museum of Ireland': (),
    'National Museum of Natural History': (),
    'National Museum of Scotland': (),
    'Nationalmuseet, Copenhagen DK': (),
    'Natural History Museum': (),
    'Natural History Museum of Los Angeles': (),
    'Natural History Museum, Basel': (),
    'Natural History Museum, Vienna': (),
    'Naturwissenschaftliche Sammlungen Winterthur': (),
    'Nicholson Museum University of Sydney': (),
    'Norfolk Museums': (),
    'Ohio History Connection': (),
    'Old Capital Museum': (),
    'Oriental Insitute University of Chicago': (),
    'Otago Museum': (),
    'Peabody Museum': (),
    'Pechersk Lavra': (),
    'Penn Museum': (),
    'Perth Museum and Art Gallery': (),
    'Queensland Museum': (),
    'REM, Mannheim, Germany': (),
    'RISD Museum': (),
    'Ratisches Museum Chur CH': (),
    'Raymond Dart Collection': (),
    'Redpath Museum': (),
    'Reiss-Engelhorn-Museen, Mannheim DE': (),
    'Rijksmuseum van Oudheden': (),
    'Robert Hull Fleming Museum': (),
    'Rosicrucian Egyptian Museum': (),
    'Royal Ontario Museum': (),
    'Rueil-Malmaison museum': (),
    'Rutgers Geology Museum on loan from New Brunswick Theological Seminary': (),
    'Ryusui-ji Dainchibou Temple': (),
    'Saint Louise Art Museum': (),
    'San Francisco Fine Arts Museum': (),
    'San Fransico State University Museum': (),
    'Schloss Gottorf, Schleswig DE': (),
    'Shemanovsky Museum': (),
    'Shrewsbury Museum & Art Gallery': (),
    'Society of Vertebrate Paleontology': (),
    'South Australia Museum': (),
    'South Australian Museum': (),
    'South Tyrol Museum of Archaeology': (),
    'Southern Baptist Theological Seminary': (),
    'Speed Art Museum': (),
    'Spurlock Museum University of Illinois': (),
    "St. Michan's Church",
    'St. Petersburg Museum of History': (),
    'Staatliche Kunstsammlungen Dresed': (),
    'State Archaeology Museum (India)': (),
    'State Museum, India': (),
    'Stiftsbibliothe, St. Gallen': (),
    'Swansea Museum U.K.': (),
    'Tasmanian Museum & Art Gallery': (),
    'Te Papa (prev. Colonial Museum)': (),
    'The Art Institute of Chicago': (),
    'Tokyo National Museum': (),
    'Torquay Museum': (),
    'Tulane': (),
    'UNKNOWN': 'UNKNOWN',
    'Ulster Museum': (),
    'Universalmuseeum Joanneum Graz AU': (),
    'Universite de Quebec a Montreal': (),
    'University Museums of Padua': (),
    'University of Alaska Musem of the ': (),
    'University of Pennsylvania Museum': (),
    'University of Pisa': (),
    'University of Richmond': (),
    'University of Sydney': (),
    'University of Western Ontario': (),
    'University of Wyoming': (),
    'Vatican Museum': (),
    'Victor Balaguer Library Museum': (),
    'Virginia Museum of Fine Arts': (),
    'Volkerkundemuseum Basel CH': (),
    'Walters Art Museum': (),
    'Wayne County Historical Museum': (),
    'Westminster College': (),
    'Williams College Musem of Art': (),
    'World Museum Liverpool': (),
    'Xingjiang Museum': (),
    'Yale Peabody': (),
    'de Young Museum': (),
}

In [78]:
sorted(str(x) for x in mummies_df.Institution.unique())

['',
 ' Fondazione Museo delle Antichità Egizie',
 '(?) Museum of Fine Arts Houston',
 '(Tomb 1 & 2)',
 '08-480.2a-c',
 '11.6',
 '52.128a-e',
 '8898*88**',
 'Academy of Natural Sciences of Drexel University',
 'Aksaray Müzesi',
 'Albany Institute of History & Art',
 'Albert Hall Museum',
 'Amasya Museum Hall of Mummies',
 'Archaeological Museum in Zagreb',
 'Archaeological Museum of Zanjan',
 'Assaraya Alhamra Museum',
 'Australia Museum',
 'Australian Museum',
 'Barnum Museum',
 'Basel University',
 'Bavarian State Archaeological Collection',
 'Berkshire Museum',
 'Bolton Museum',
 'Boston Museum of Fine Arts',
 'British Museum',
 'Brooklyn Museum',
 'Buffalo Museum of Science',
 'Burke Museum',
 'CG24100',
 'CG33281',
 'Cairo Museum The Virtual Eqyptian Museum',
 'Caldwell Heritage Museum',
 'Canadian Museum of Civilization',
 'Canterbury Museum, NZ',
 'Carnegie Museum',
 'Centre for the Study of the Arctic;Shemanovsky Science Museum and Exhibition Centre',
 'Chatham-Kent Museum',
 '

In [44]:
mummies_df.Institution.value_counts()

Medelhavsmuseet                                                                        83
Egyptian Museum of Antiquities                                                         41
Leiden Museum                                                                          30
Egypt Centre - Canolfan Eiffraidd, Swansea                                             29
Manchester Museum                                                                      29
World Museum Liverpool                                                                 25
Musee Royaux, Brussels                                                                 24
Museum of Fine Arts Boston                                                             21
Australian Museum                                                                      20
The Metropolitan Museum of Art                                                         17
National Museum of Scotland                                                            12
Myers Muse